## import overpy

In [1]:
import overpy
api = overpy.Overpass()

## download file from overpass api 

In [2]:
'''

result = api.query("""
   <osm-script timeout="1800" element-limit="100000000">
  <union>
    <area-query ref="3600007444"/>
    <recurse type="node-relation" into="rels"/>
    <recurse type="node-way"/>
    <recurse type="way-relation"/>
  </union>
  <union>
    <item/>
    <recurse type="way-node"/>
  </union>
  <print mode="body"/>
</osm-script>
    """)
    
    
'''

'\n\nresult = api.query("""\n   <osm-script timeout="1800" element-limit="100000000">\n  <union>\n    <area-query ref="3600007444"/>\n    <recurse type="node-relation" into="rels"/>\n    <recurse type="node-way"/>\n    <recurse type="way-relation"/>\n  </union>\n  <union>\n    <item/>\n    <recurse type="way-node"/>\n  </union>\n  <print mode="body"/>\n</osm-script>\n    """)\n    \n    \n'

## or load file from disk

In [3]:
file = open("Paris", encoding='utf-8')
result = api.parse_xml(file.read(), encoding='utf-8', parser=None)

## get all transport line, 

In [13]:
transport = 'train'
#tranport = 'bus'
#tranport = 'subway'

In [14]:
relation_transport= {}
for relation in result.relations:
    for member in relation.members:
        if 'route' in relation.tags.keys():
            if transport in relation.tags['route']:
                relation_transport [relation.id]= relation.tags['name']

In [15]:
for key in relation_transport:
    print (relation_transport[key])

RER E2 : Haussmann - Saint-Lazare → Chelles - Gournay
Transilien H
Lyria : Paris - Lausanne
Transilien R (KUMO) : Gare de Lyon → Montereau
Transilien P (PIMO) : Château-Thierry → Gare de l'Est
TGV 310 : Paris -- Le Croisic
IC 501 : Paris - Nevers - Clermont-Ferrand
RER D - Branche Nord, sens Sud
RER D - Branche Nord, sens Nord
Thalys: Paris → Bruxelles → Amsterdam
RER D - Branche Juvisy, sens Sud → Nord
RER D - Branche Combs la Ville Quincy, sens Nord → Sud
RER D - Branche Juvisy, sens Nord → Sud
RER D - Branche Combs la Ville Quincy, sens Sud → Nord
TER 4.2 : Paris − Chartres − Nogent-le-Rotrou − Le Mans
TGV 308 : Paris -- Brest
TER K13 : Paris Gare du Nord → Saint-Quentin → Maubeuge
Transilien K (AEKY) : Crépy-en-Valois → Gare du Nord
Paris => Rouen => Le Havre
TER 2.1 : Tours - Blois - Orléans (- Paris)
TER 23 : Le Mans − Nogent le Rotrou − Paris
TGV 101 : Francfort → Paris Est
Paris → Trouville-Deauville
RER A4 : Poissy → Marne-la-Vallée - Chessy
RER A4 : Cergy-le-Haut → Marne-la-V

## get relations from transport id

In [16]:
transport_to_relation= {}
for relation in result.relations:
    if 'route' in relation.tags.keys():
        if transport in relation.tags['route']:
            for member in relation.members:
                if(member.role=='stop'):
                    if member.ref in transport_to_relation :
                        transport_to_relation [member.ref].append(relation.id)
                    else:
                        transport_to_relation [member.ref] = [relation.id]
                    

In [17]:
#transport_to_relation

In [18]:
for key in transport_to_relation:
    node = result.get_nodes(key)
    print (node)
    if (node):
        print (node[0].lat, node[0].lon)
    break

[<overpy.Node id=9532923969 lat=48.8816622 lon=2.3592798>]
48.8816622 2.3592798


## get relation from way id 

In [19]:
way_to_relation= {}
for relation in result.relations:
    if 'route' in relation.tags.keys():
        if transport in relation.tags['route']:
            for member in relation.members:
                if(member.role==None):
                    if member.ref in way_to_relation :
                        way_to_relation [member.ref].append(relation.id)
                    else:
                        way_to_relation [member.ref] = [relation.id]

##  create point layer by using Fiona

In [20]:
import fiona

In [21]:
# define schema
schema = {
    'geometry':'Point',
    'properties':[('relation_id','int'),('relation_name','str') ,('osm_id','int')]
}

file = transport + ".shp"
print (file)
pointShp = fiona.open(file, mode='w', driver='ESRI Shapefile',schema = schema, crs = "EPSG:4326")
#iterate over each row in the dataframe and save record
for key in transport_to_relation:
    node = result.get_nodes(key)
    if (node):
        for i in transport_to_relation[key]:
            rowDict = {
                'geometry' : {'type':'Point','coordinates': (node[0].lon,node[0].lat)},
                'properties': {  'relation_id': i , 'relation_name' : relation_transport[i] , 'osm_id' : node[0].id},
            }
            pointShp.write(rowDict)
    
#close fiona object
pointShp.close()

train.shp


## point.shp

| relation_id| relation_name | osm_id |
| --- | --- | --- |
| 1255723 | Bus 29 : Porte de Montempoivre ? Gare Saint-Lazare | 2042462222 |
| 1255729 | Bus 29 : Gare Saint-Lazare ? Porte de Montempoivre | 2042462222 |

## find the nearly building for each point, add the info to the building

hub_distance{ 'FIELD' : 'osm_id', 'HUBS' : 'batiment_converted.shp', 'INPUT' : 'point.shp', 'OUTPUT' : 'TEMPORARY_OUTPUT', 'UNIT' : 0 }

|relation_id|relation_name|osm_id|code|fclass|name|hubname|hubdist|
|---|---|---|---|---|---|---|---|
|1254454	|Bus 20 : Porte des Lilas ? Louison Bobet	|1821967163	|5621	|bus_stop	|Grands Boulevards	|5013364	|9.516719518748575|
|1255724	|Bus 39 : Gare du Nord ? Frères Voisin (Issy-les-Moulineaux)	|1821967163	|5621	|bus_stop	|Grands Boulevards	|5013364	|9.516719518748575|